In [1]:
from __future__ import print_function
import h5py
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input, Concatenate
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import plot_model
from keras.models import Model
from keras import backend as K


from keras.utils import multi_gpu_model
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import copy
import tensorflow as tf

import gc

import time

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
%matplotlib inline

Using TensorFlow backend.
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\mrathbun2018\.con

In [2]:
hdf5_path = 'Data/dataset.hdf5'
subtract_mean = True

hdf5_file = h5py.File(hdf5_path, "r")

if subtract_mean:
    mm = hdf5_file["train_mean"][...,0]
    mm = mm[np.newaxis, ...]

data_num = hdf5_file["train_flow"].shape[0]
    
num_classes = 2
epochs = 30

flow_rows, flow_cols = 298, 17

x_train = hdf5_file["train_flow"][...,0]
if subtract_mean:
    x_train -= mm

y_train = hdf5_file["train_labels"][:,...]
hdf5_file.close()
hdf5_path = 'Data/dataset-IoT.hdf5'
hdf5_file = h5py.File(hdf5_path, "r")


x_test = hdf5_file["IoT_flow"][...,0]
if subtract_mean:
    x_test -= mm

y_test = hdf5_file["labels"][:,...]

hdf5_file.close()

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
d_class_weights = dict(enumerate(class_weights))

#define number of packets used to train for each device
packets = 210
x_train = x_train[:,:packets,:]
x_test = x_test[:,:packets,:]

input_shape = (x_train.shape[1], x_train.shape[2])
    
    
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [3]:
AccScores = []
Times = []
LossScores = []
#AucScores = []
#F1Scores = []
#PrecisionScores = []
t2 = time.time()

for i in range (1,50):
    size = 64
    nb_filters = 32
    num_classes = 2
    epochs=30
    activations= "tanh"
    maxlen=packets
    dropout = 0.24295335733172563
    batch_size = 512
    pool_size = 3
    lr = 0.0012168203727788032
    adam = keras.optimizers.Adam(lr=lr)

    optim = adam

    layers = 3
    model = Sequential()
    model.add(Conv1D(filters=nb_filters,kernel_size=size,input_shape=(input_shape),padding="valid",activation=activations,strides=1))
    for i in range(layers-1):
        model.add(Conv1D(filters=nb_filters,kernel_size=size,padding="valid",activation=activations,strides=1))
        
    model.add(MaxPooling1D(pool_size= pool_size))
    model.add(Flatten())
    model.add(Dropout(dropout))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(num_classes, activation='softmax'))
    #model.summary()
    try:
        model = multi_gpu_model(model, gpus = 4)
    except:
        pass
        
    model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])
    t0 = time.time()
    model.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, verbose=0, class_weight=class_weights, shuffle=True)
    scores = model.evaluate(x_test, y_test, verbose=0)
    t1 = time.time()
    total_run = t1-t0
    Times.append(total_run)
    print("%s: %.5f%%" % (model.metrics_names[1], scores[1]*100))
    print("%s: %.5f%%" % (model.metrics_names[0], scores[0]))
    print('Fit and Evaluate Time:',total_run)
    AccScores.append(scores[1] * 100)
    LossScores.append(scores[0])
    
    #y_pred = model.predict(x_test)
    #yy_test = [np.argmax(i) for i in y_test]

    #yy_pred = [np.argmax(i) for i in y_pred]
  
    #new = np.vstack([yy_test,yy_pred])
    #print(classification_report(yy_test, yy_pred, digits = 5 ))
    #f1 = f1_score(yy_test, y_pred)
    #F1Scores.append(f1)
    
    #ap = average_precision_score(yy_test, y_pred[:,1])
    #PrecisionScores.append(ap)
    

    """

    fpr_keras, tpr_keras, thresholds_keras = roc_curve(yy_test, y_pred[:,0],pos_label=0)
    auc_keras = auc(fpr_keras, tpr_keras)
    AucScores.append(auc_keras)
    """
    del model
    gc.collect()
    K.clear_session()
    gc.collect()

    
print("%.5f%% (+/- %.5f%%)" % (np.mean(AccScores), np.std(AccScores)))
print("%.5f%% (+/- %.5f%%)" % (np.mean(LossScores), np.std(LossScores)))
print("%.5f%% (+/- %.5f%%)" % (np.mean(Times), np.std(Times)))
#print("%.5f%% (+/- %.5f%%)" % (np.mean(AucScores), np.std(AucScores)))
#print("%.5f%% (+/- %.5f%%)" % (np.mean(F1Scores), np.std(F1Scores)))
#print("%.5f%% (+/- %.5f%%)" % (np.mean(PrecisionScores), np.std(PrecisionScores)))
t3 = time.time()
total_time = t3-t2
print('Time to do everything:',total_time)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

accuracy: 88.66886%
loss: 0.25935%
Fit and Evaluate Time: 74.26235580444336
accuracy: 89.20483%
loss: 0.26116%
Fit and Evaluate Time: 67.0567262172699
accuracy: 89.67202%
loss: 0.25677%
Fit and Evaluate Time: 66.8016037940979
accuracy: 89.10997%
loss: 0.25671%
Fit and Evaluate Time: 67.26354169845581
accuracy: 89.35898%
loss: 0.25011%
Fit and Evaluate Time: 66.44705939292908
accuracy: 88.41037%
loss: 0.25855%
Fit and Evaluate Time: 67.28670525550842
accuracy: 89.13842%
loss: 0.25442%
Fit and Evaluate Time: 66.3300633430481
accuracy: 89.31629%
loss: 0.25216%
Fit and Evaluate Time: 67.07463574409485
accuracy: 89.03408%
loss: 0.25439%
Fit and Evaluate Time: 66.7643187046051
accuracy: 89.27597%
loss: 0.25434%
Fit and Evaluate Time: 66.8688395023346
accuracy: 89.11471%
loss: 0.25019%
Fit and Evaluate Time: 66.01121616363525
accuracy: 89.49890%
loss: 0.25187%
Fit and Evaluate Time: 67.089039325714

In [4]:
AccScores

[88.66886496543884,
 89.20482993125916,
 89.67201709747314,
 89.10996913909912,
 89.35897946357727,
 88.41036558151245,
 89.13842439651489,
 89.31629061698914,
 89.0340805053711,
 89.27597403526306,
 89.11470770835876,
 89.49889540672302,
 88.45542669296265,
 89.28782939910889,
 89.82142210006714,
 88.84198665618896,
 88.06886672973633,
 88.58111500740051,
 89.28782939910889,
 88.87755870819092,
 89.1692578792572,
 88.88704180717468,
 89.19296860694885,
 89.18111324310303,
 88.68546485900879,
 89.44435119628906,
 89.09573554992676,
 88.96530270576477,
 88.42697143554688,
 89.2664909362793,
 88.38428258895874,
 89.20008540153503,
 89.06253576278687,
 88.86095881462097,
 88.81115317344666,
 88.73526453971863,
 88.3866548538208,
 88.80167007446289,
 88.7044370174408,
 88.67360949516296,
 88.83486986160278,
 88.23250532150269,
 88.79929780960083,
 88.13527226448059,
 88.86570334434509,
 89.15265202522278,
 89.29257392883301,
 88.97241950035095,
 88.99850845336914]

In [5]:
LossScores

[0.2593475821615891,
 0.26116378010725777,
 0.2567737456779968,
 0.2567111481236697,
 0.2501116290972636,
 0.2585457458460996,
 0.2544196022006854,
 0.2521558454804928,
 0.2543896660127424,
 0.2543429039404739,
 0.2501858270200533,
 0.25187424611455983,
 0.26733159638007825,
 0.25385067766291075,
 0.2517715325413796,
 0.2582349775403517,
 0.25948571404496745,
 0.2504973009822039,
 0.25933104252607175,
 0.25306688412392453,
 0.2559747727874105,
 0.25601118781132126,
 0.24947939812076902,
 0.25443345032098563,
 0.2583245969544587,
 0.25188562783665547,
 0.26751395496447783,
 0.2571796265121294,
 0.2574381415148701,
 0.24575423957073633,
 0.25803214582539574,
 0.266049307009064,
 0.25426672133652334,
 0.2574159592150819,
 0.25426651996323385,
 0.26501452281481314,
 0.26348079646965533,
 0.2552743505353545,
 0.25850842117868866,
 0.2564336486093789,
 0.24781282758590922,
 0.27097763357065197,
 0.256472502046628,
 0.26448322193948043,
 0.2631979433819961,
 0.25641162460143113,
 0.2467447275

In [6]:
Times

[74.26235580444336,
 67.0567262172699,
 66.8016037940979,
 67.26354169845581,
 66.44705939292908,
 67.28670525550842,
 66.3300633430481,
 67.07463574409485,
 66.7643187046051,
 66.8688395023346,
 66.01121616363525,
 67.08903932571411,
 66.59796929359436,
 66.98265361785889,
 65.60863494873047,
 66.25834083557129,
 66.64534139633179,
 65.94114470481873,
 66.49225449562073,
 67.35362958908081,
 67.0973596572876,
 67.0540030002594,
 66.61700654029846,
 66.32690620422363,
 67.19586396217346,
 67.3721239566803,
 66.1276957988739,
 67.25383758544922,
 66.28333926200867,
 66.19804215431213,
 66.41659736633301,
 66.43506264686584,
 66.35014605522156,
 66.7872486114502,
 65.90563344955444,
 67.14626789093018,
 66.01272964477539,
 66.72834944725037,
 65.8475399017334,
 66.3837378025055,
 66.09962463378906,
 66.11366510391235,
 66.30084276199341,
 67.31245541572571,
 65.75047826766968,
 66.26911044120789,
 66.85990571975708,
 66.54003596305847,
 66.7682957649231]